# Using MLflow models in batch deployments

In [6]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import BatchEndpoint, BatchDeployment, Model, AmlCompute, Data
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential

Class SystemCreatedStorageAccount: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SystemCreatedAcrAccount: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [7]:
subscription_id = "<subscription>"
resource_group = "<resource-group>"
workspace = "<workspace>"
   
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

Class RegistryOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Let's verify if the model we want to deploy, `heart-classifier`, is registered in the model registry. If not, we will register it from a local version we have in the repository:

In [8]:
model_name = "heart-classifier"
model_local_path = "heart-classifier-mlflow/model"

In [9]:
if not any(filter(lambda m : m.name == model_name, ml_client.models.list())):
    print(f"Model {model_name} is not registered. Creating...")
    model = ml_client.models.create_or_update(
        Model(name=model_name, path=model_local_path, type=AssetTypes.MLFLOW_MODEL)
    )

Let's get the model:

In [10]:
model = ml_client.models.get(name=model_name, label="latest")

First, let's create the endpoint that is going to host the batch deployments. Remember that each endpoint can host multiple deployments at any time, however, only one of them is the default one:

In [11]:
endpoint = BatchEndpoint(
    name="heart-classifier-batch", 
    description="A heart condition classifier for batch inference",
)

In [14]:
ml_client.batch_endpoints.begin_create_or_update(endpoint)

Batch endpoints can run on any Azure ML compute that already exists in the workspace. That means that multiple batch deployments can share the same compute infrastructure. In this example, we are going to work on an AzureML compute cluster called `cpu-cluster`. Let's verify the compute exists on the workspace or create it otherwise.

In [12]:
compute_name = "cpu-cluster"
if not any(filter(lambda m : m.name == compute_name, ml_client.compute.list())):
    print(f"Compute {compute_name} is not created. Creating...")
    compute_cluster = AmlCompute(name=compute_name, description="amlcompute", min_instances=0, max_instances=5)
    ml_client.begin_create_or_update(compute_cluster)

Compute may take time to be created. Let's wait for it:

In [13]:
from time import sleep

print("Waiting for compute", end="")
while ml_client.compute.get(name=compute_name).provisioning_state == "Creating":
    sleep(1)
    print(".", end="")

print(" [DONE]")

Waiting for compute [DONE]


Let's create a deployment under the given endpoint.

In [14]:
from azure.ai.ml.constants import BatchDeploymentOutputAction
from azure.ai.ml.entities import BatchRetrySettings

deployment = BatchDeployment(
    name="classifier-xgboost",
    description="A heart condition classifier based on XGBoost",
    endpoint_name=endpoint.name,
    model=model,
    compute=compute_name,
    instance_count=2,
    max_concurrency_per_instance=2,
    mini_batch_size=10,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=300),
    logging_level="info",
)

In [17]:
ml_client.batch_deployments.begin_create_or_update(deployment)

Let's update the default deployment name in the endpoint:

In [38]:
endpoint.defaults.deployment_name = deployment.name
ml_client.batch_endpoints.begin_create_or_update(endpoint)

We can see the endpoint URL as follows:

In [20]:
endpoint.scoring_uri

'https://heart-classifier-batch.eastus.inference.ml.azure.com/jobs'

## Testing the endpoint

Once the deployment is created, it is ready to recieve jobs. Let's first register a data asset so we can run the job against it. This data asset is a folder containing multiple CSV files that we want to process in parallel using the batch endpoint we just created.

In [21]:
data_path = "heart-classifier-mlflow/heart-dataset/"
dataset_name = "heart-dataset-unlabeled"

heart_dataset_unlabeled = Data(
    path=data_path,
    type=AssetTypes.URI_FOLDER,
    description="An unlabeled dataset for heart classification",
    name=dataset_name,
)

ml_client.data.create_or_update(heart_dataset_unlabeled)

Data({'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_folder', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'heart-dataset-unlabeled', 'description': 'An unlabeled dataset for heart classification', 'tags': {}, 'properties': {}, 'id': '/subscriptions/18522758-626e-4d88-92ac-dc9c7a5c26d4/resourceGroups/Analytics.Aml.Experiments.Workspaces/providers/Microsoft.MachineLearningServices/workspaces/aa-ml-aml-workspace/data/heart-dataset-unlabeled/versions/6', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/santiagxf-cpu/code/Users/fasantia/azureml-examples/sdk/python/endpoints/batch', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f714eb16c10>, 'serialize': <msrest.serialization.Serializer object at 0x7f714e9dcaf0>, 'version': '6', 'latest_version': None, 'path': 'azureml://subscriptions/18522758-626e-4d88-92ac-dc9c7a5c26d4/resourcegroups/Analytics.Aml.Ex

In [22]:
heart_dataset_unlabeled = ml_client.data.get(name="heart-dataset-unlabeled", label="latest")

Let's use this data as an input for the job:

In [23]:
from azure.ai.ml import Input

input = Input(type=AssetTypes.URI_FOLDER, path=heart_dataset_unlabeled.id)

In [13]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint.name,
    input=input
)

You can use the returned job object to check the status of the job:

In [18]:
ml_client.jobs.get(job.name)

Experiment,Name,Type,Status,Details Page
heart-classifier-batch,39e1d9d3-a567-40dc-884c-fcad73630f49,pipeline,Running,Link to Azure Machine Learning studio


Notice how we are not indicating the deployment name in the invoke operation. That's because the endpoint automatically routes the job to the default deployment. Since our endpoint only has one deployment, then that one is the default one. You can target an specific deployment by indicating the argument/parameter `deployment_name`.

In [ ]:
job = ml_client.batch_endpoints.invoke(
    deployment_name=deployment.name,
    endpoint_name=endpoint.name,
    input=input
)

### Overwriting deployment degree of parallelism in jobs

Batch endpoints allow you to overwrite some of the parameters of the batch deployment. You can use this to control the degree of parallelization in your job depending to the size of the data input you are using:

In [ ]:
job = ml_client.batch_endpoints.invoke(
    deployment_name=deployment.name,
    endpoint_name=endpoint.name,
    input=input,
    params_override=[{"mini_batch_size": "20"}, {"compute.instance_count": "1"}],
    
)

## Exploring the results

In [6]:
ml_client.jobs.download(name=job.name, download_path=".", output_name='score')

In [2]:
with open('named-outputs/score/predictions.csv', 'r') as f:
    data = f.read()

In [4]:
from ast import literal_eval
import pandas as pd

score = pd.DataFrame(literal_eval(data.replace('\n', ',')), columns=['file', 'prediction'])
score

,file,prediction
0,heart-unlabeled-0.csv,0
1,heart-unlabeled-0.csv,1
2,heart-unlabeled-0.csv,0
3,heart-unlabeled-0.csv,0
4,heart-unlabeled-0.csv,0
...,...,...
298,heart-unlabeled-2.csv,0
299,heart-unlabeled-2.csv,1
300,heart-unlabeled-3.csv,1
301,heart-unlabeled-3.csv,1
